### imports and load data

In [18]:
import pandas as pd
import numpy
import datetime
import math
import io

from google.colab import files
uploaded = files.upload()

In [19]:
df = pd.read_excel(io.BytesIO(uploaded['calculation.xlsm']),sheet_name='from_calc_sheet')
df.head(3)

,id,DoB,DoT,amt,province,Pensionable Service***:
0,1,1996-02-24 00:00:00,1997-01-30,123.45,Ontario,1.04650
1,2,1997-05-22 00:00:00,2003-08-19,1234.34,Quebec,22.44558
2,3,1999-04-17 00:00:00,1997-08-09,23234.56,Newfoundland and Labrador,9.75409


### assignments and functions

In [20]:
lookup = {1: 'janvier', 2: 'février', 3: 'mars', 4: 'avril', 5: 'mai',
          6: 'juin', 7: 'juillet', 8: 'août', 9: 'septembre', 10: 'octobre', 11: 'novembre', 12: 'décembre'}

fr_prov_dict = {'British Columbia': 'Colombie-Britannique',
                'Did Not Work in ON or NS': "N'a pas travaillé en Ontario ou en Nouvelle-Écosse",
                "New Brunswick": 'Nouveau-Brunswick',
                'Nova Scotia': 'Nouvelle-Écosse',
                'Quebec': 'Québec',
                'Newfoundland and Labrador': 'Terre-Neuve-et-Labrador',
                'Newfoundland':'Terre-Neuve-et-Labrador',
                'Prince Edward Island': 'Île-du-Prince-Édouard',
                'Northwest Territories': 'Les Territoires du Nord-Ouest',
                'Defined Contribution': 'Cotisations Déterminée'}


def translate_provinces(t_prov_str):
    for key in fr_prov_dict.keys():
        t_prov_str = t_prov_str.replace(key, fr_prov_dict[key])

    return t_prov_str


def conv_date(t_date):
    if type(t_date) != datetime.datetime and type(t_date) != pd._libs.tslibs.timestamps.Timestamp:
        return ""
    else:
        return_str = str(t_date.day) + ' ' + \
            lookup[t_date.month] + ' ' + str(t_date.year)
        return return_str


def fix_round(num, position=0):
    '''fixes rounding "error" in py - may not be accurate w/ extremely precise numbers such as 0.49999999999999998

        params:
        num (float/int): number to round
        position (int): default is 0

        returns rounded result

    '''

    if type(num) != float and type(num) != int and type(num) != numpy.float:
        return 0.00

    multiplier = 10**position
    num = multiplier * num

    try:
        after_decimal = str(num).split('.')[1]
    except:
        return (num/multiplier)

    if after_decimal == '5':
        if num > 0:
            x = math.ceil(num)
            return (round((x)/multiplier, position))
        if num < 0:
            x = math.floor(num)
            return (round((x)/multiplier, position))
    else:
        return (round(num/multiplier, position))


def conv_curr(t_amt, dec_places=2, dollar_sign=True):
    t_str = str(fix_round(t_amt, dec_places))
    t_list = t_str.split('.')

    before_dec = t_list[0]

    before_dec_list = [before_dec[0:len(before_dec) % 3]] + [before_dec[i:i+3]
                                                             for i in range(len(before_dec) % 3, len(before_dec), 3)]

    before_dec_str = ' '.join(before_dec_list)

    while len(t_list[1]) < dec_places:
        t_list[1] += '0'

    if dollar_sign == True:
        return_str = before_dec_str + ',' + t_list[1] + ' $'
    else:
        return_str = before_dec_str + ',' + t_list[1]
        
    return return_str.strip()


### user input

In [21]:
date_col_list_input = ['today',
                        'Member’s Date of Birth',
                        'Date of Hire',
                        'Date of Entry',
                        'Spouse’s Date of Birth',
                        'norm_ret_date',
                        'eurd',
                        'calc_date',
                        'DB Date of Termination']
curr_col_list_input = ['best_avg_earnings',
                        'earliest_unreduced_amt', 'op3']
round_to_dec_list_input = [
    'age_at_actual_ret_date', 'Total Credited Service']
prov_col_list_input = ['*** Final Province of Employment (DB or DC) ***']

### run and output

In [22]:
for col in round_to_dec_list_input:
    df[col] = df[col].fillna(0)
    df[col] = df[col].apply(conv_curr, dec_places=5, dollar_sign=False)

for col in curr_col_list_input:
    df[col] = df[col].fillna(0)
    df[col] = df[col].apply(conv_curr)

for col in date_col_list_input:
    df[col] = df[col].apply(conv_date)

for col in prov_col_list_input:
    df[col] = df[col].fillna("")
    df[col] = df[col].apply(translate_provinces)

df.to_excel('test_out.xlsx', index=False)
files.download('test_out.xlsx')

C:\Users\ronal\AppData\Local\Temp/ipykernel_10648/2518931237.py:44: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(num) != float and type(num) != int and type(num) != numpy.float:
